<a href="https://colab.research.google.com/github/Pepe-Padilla/deep-learning/blob/main/Modelo1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo1D - DeepLearning


### Indice
1. Introducción
2. Importación y Normalización de datos
3. Entrenamiento
4. Gráfica de Perdidas
5. Test
6. Concluciones

##  1. Introducción

**Práctica DL** - Ejercicio de Bootcamp Inteligencia Artificial Full Stack Edición III

Este proyecto es un entregable para la práctica del Master Bootcamp Inteligencia Artificial Full Stack Edición III realizado por el centro de formación [@Keepcoding](https://github.com/KeepCoding)

---

El objetivo de este trabajo consiste en resolver un problema del mundo real empleando para ello técnicas vistas durante las sesiones de dicho módulo. En concreto, se trabajará en predecir la condición médica sufrida por una cohorte de pacientes a partir de las diferentes fuentes de información disponibles (imágenes y datos tabulares).

Las imagenes deben entrenar y obtener una de los siguientes respuestas:

| Enfermedad | Descripción | Código | Indice en salida |
|------------|-------------|--------|------------------|
| Actinic keratoses y carcinoma de células escamosas | Tipo de cáncer de piel | akiec | 0 |
| Nevus melanocítico | Lesión benigna común | nv | 1 |
| Melanoma | Tipo de cáncer de piel agresivo | mel | 2 |
| Lesiones benignas de queratosis | Incluyen lentigo solar y queratosis seborreica | bkl | 3 |
| Dermatofibroma | Lesión benigna del tejido fibroso | df | 4 |
| Vasculares | Lesiones vasculares como hemangiomas | vasc | 5 |
| Lesión de células basales | Un tipo de cáncer de piel menos agresivo que el melanoma | bcc | 6 |

---



## 2. Importación y Normalización de datos

### 2.1 Imports

In [26]:
# 1. Primero las importaciones que necesitaremos
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

### 2.2 Obtención de datos

In [27]:
# 2. Obtención de datos
df = pd.read_csv("/content/HAM10000_metadata.csv", sep=",")


### 2.3 Inspección de elementos

In [28]:
# 3. Inspeccionamos y normalización
print(df.head())
print(df.info())
print(df.describe())

# Primero busquemos nulls en las columnas que valgan la pena
print(df.isnull().sum())

# Solo age tiene null, solo se me ocurre rellenar con la mediana
df["age"] = df["age"].fillna(df["age"].median())
df['age'] = df['age'] / 100  # Escalamos considerando 100 como el máximo teórico

# para evitar problemas después, convertir el tipo de age a un float compatible para Tensor
df['age'] = df['age'].astype(np.float32)

# Convertimos las variables 'sex' y 'localization' a categorías one-hot encoding
df = pd.get_dummies(df, columns=['sex', 'localization'])


     lesion_id      image_id   dx dx_type   age   sex localization
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
dtypes: float64(1), object(6)
memory usage: 547.8+ KB
None
               age
count  9958.000000
mean  

### 2.4 Separación de datos

In [29]:
# 4. Separamos datos
x = df.drop(['lesion_id', 'image_id', 'dx', 'dx_type'], axis=1)
y = pd.get_dummies(df['dx'], columns=['dx'])

x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.20)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, shuffle=True, test_size=0.15)

### 2.5 Verificación

In [41]:
# 5. Verificación de normalización
print(x_train.info())
print(y_train.info())
print(x_train.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 6810 entries, 1508 to 3452
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           6810 non-null   float32
 1   sex_female                    6810 non-null   bool   
 2   sex_male                      6810 non-null   bool   
 3   sex_unknown                   6810 non-null   bool   
 4   localization_abdomen          6810 non-null   bool   
 5   localization_acral            6810 non-null   bool   
 6   localization_back             6810 non-null   bool   
 7   localization_chest            6810 non-null   bool   
 8   localization_ear              6810 non-null   bool   
 9   localization_face             6810 non-null   bool   
 10  localization_foot             6810 non-null   bool   
 11  localization_genital          6810 non-null   bool   
 12  localization_hand             6810 non-null   bool   
 13  local

## 3. Entrenamiento

In [43]:
# Imports necesarios
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

# Hiper-parámetros de nuestra red
lr = 0.005
n_epochs = 50
batch_size = 128

# Implementamos la red empleando Keras
model = Sequential() # Instancia de modelo API secuencial
model.add(Flatten()) # Estiramos los datos en forma de vector como entrada a nuestro Perceptrón Simple
model.add(Dense(10, input_shape=(19,), activation="softmax")) # Construimos nuestro Perceptrón simple con una única capa Dense

# Compilamos y entrenamos el modelo SGD
print("[INFO]: Entrenando red neuronal...")
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr), metrics=["accuracy"])
H = model.fit(x_train.values, y_train, validation_data=(x_val.values, y_val), epochs=n_epochs, batch_size=batch_size)

# Evaluando el modelo de predicción
print("[INFO]: Evaluando red neuronal...")
predictions = model.predict(x_test, batch_size=batch_size)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))



[INFO]: Entrenando red neuronal...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: object

## 4. Gráfica de Perdidas

In [ ]:
# veamos nuestra función de pérdidas con respecto a las épocas ejecutadas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, n_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, n_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, n_epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, n_epochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

## 5. Test

In [ ]:
# comprobamos lo que ha aprendido nuestra red
pred = tf.nn.softmax(tf.matmul(x_test, W) + b)
correct_prediction =   tf.equal(tf.argmax(pred, 1), tf.argmax(y_test, 1))

# calculamos el accuracy (precisión)
accuracy =  tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", accuracy.numpy())

## 6. Concluciones

Aquí van mis concluciones